In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import datetime
from gee_scripts.get_sources import get_explanatory_composite
from gee_scripts.parameters import explain_vars
from gee_scripts.get_sources import get_s1_dates
from gee_scripts.gee import export_classifier
from gee_scripts.directories import get_export_folder
from gee_scripts import init_ee

import ee


Earth Engine initialized successfully, with ee-indonesia-gwl


## 1. Train the classifier in GEE and save the model as asset

In [3]:
# This table contains the training data for the model
training_data = ee.FeatureCollection("projects/ee-indonesia-gwl/assets/all_training_data_with_extra_and_locations_and_precipSum")

In [4]:
best_kalimantan_phus = [297, 351, 352, 256, 340]
training_data = training_data.filter(ee.Filter.inList("phu_id", best_kalimantan_phus))
training_data.size().getInfo()

3575

In [5]:
model_gee_id = export_classifier(training_data, "kalimantan_best_phus")
model_gee_id

Exporting model kalimantan_best_phus with 3575 samples
Exported model projects/ee-indonesia-gwl/assets/gwl-modeling/models/RandomForest_kalimantan_best_phus_trees_250


'projects/ee-indonesia-gwl/assets/gwl-modeling/models/RandomForest_kalimantan_best_phus_trees_250'

## 2. Estimate using the trained model

https://code.earthengine.google.com/1fd31fe53d5a8cf8b812552f901325a0

## 2.1. Set parameters

In [6]:
target_phu = ee.FeatureCollection("projects/ee-indonesia-gwl/assets/All_phus_numbered").filter(
    ee.Filter.eq("phu_id", 297)).first(\
).geometry().bounds()

# This lie will take some time to run, because it will do a GEE request
all_s1_dates = get_s1_dates(aoi=target_phu)

In [8]:
all_str_date = [
    datetime.datetime.fromtimestamp(date/1000).strftime('%Y-%m-%d') for date in all_s1_dates
]

In [20]:
dates_df = pd.DataFrame(all_str_date, columns=["date"])
dates_df["date"] = pd.to_datetime(dates_df["date"])

# Only keep the dates after 2019
dates = dates_df[dates_df["date"] > "2019-01-01"]
len(dates)

all_str_date = dates["date"].dt.strftime('%Y-%m-%d').tolist()
len(all_str_date)

432

In [21]:
def export_estimation(model_name, aoi_name, ee_aoi, str_date, ee_classifier, ):
    """Export the estimated GWL image for a given model and target date."""

    # Get explanatory composite closest to target date
    image = get_explanatory_composite(
        target_date=str_date, 
        ee_region=ee_aoi, 
        max_days_offset=1
    ).select(explain_vars)

    output_image_name = f"{model_name}_{aoi_name}_{str_date}"
    estimated_image = image.select(explain_vars).classify(ee_classifier).set({"model": model_name, "date": str_date})

    # create export task
    task = ee.batch.Export.image.toAsset(
        **{
            "image": estimated_image,
            "description": output_image_name,
            "assetId": str(get_export_folder(output_folder=f"estimation/{model_name}")/output_image_name),
            "region": ee_aoi,
            "scale": 100,
        }y_pred_test

In [24]:
aoi_name = "PHU_297"
ee_classifier = ee.Classifier.load(model_gee_id)
model_name = "kalimantan_best_phus"
phus = ee.FeatureCollection("projects/ee-indonesia-gwl/assets/All_phus_numbered")

tasks = [
    export_estimation(model_name, aoi_name, target_phu, target_date, ee_classifier) 
    for target_date 
    in all_str_date[10:]
]

KeyboardInterrupt: 

In [23]:
[task.start() for task in tasks]

[None, None, None, None, None, None, None, None, None, None]